# Haiku aus dem Nationalrat

Der korrekte Plural von Haiku ist Haiku.  
Der Verständlichkeit halber wird im Code aber *Haikus* verwendet.

protokolle laden und flatten

In [1]:
import json
import pandas as pd

with open("woswormeileistung/data/sessions.json") as f:
    sessions = json.load(f)

wortmeldungen = pd.json_normalize(
    sessions,
    record_path=["sections"],
    meta=["period", "sessionNumber", "date"],
)

wortmeldungen = wortmeldungen[["period", "sessionNumber", "date", "speaker", "text"]]
wortmeldungen = wortmeldungen.rename(columns={"sessionNumber": "session"})

wortmeldungen.head()

,period,session,date,speaker,text
0,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Meine sehr ge...
1,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Meine sehr ge...
2,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Der Herr Bund...
3,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Die Amtlichen...
4,XXVII,276,2024-09-18T00:00:00,88386,Präsident Mag. Wolfgang Sobotka: Ich darf beka...


sprecher:in label aus text entfernen

In [2]:
wortmeldungen["text"] = wortmeldungen["text"].str.split(": ", n=1).str[1]
wortmeldungen.head()

,period,session,date,speaker,text
0,XXVII,276,2024-09-18T00:00:00,88386,Meine sehr geehrten Damen und Herren Abgeordne...
1,XXVII,276,2024-09-18T00:00:00,88386,Meine sehr geehrten Damen und Herren auf der G...
2,XXVII,276,2024-09-18T00:00:00,88386,Der Herr Bundespräsident hat mit Entschließung...
3,XXVII,276,2024-09-18T00:00:00,88386,Die Amtlichen Protokolle der 272. und der 273....
4,XXVII,276,2024-09-18T00:00:00,88386,"Ich darf bekannt geben, dass von der Bundeswah..."


dev mode: probelauf auf 1000 texte

In [3]:
wortmeldungen = wortmeldungen.sample(1000)

haiku finden

In [4]:
import spacy
import pyphen
import re
from tqdm import tqdm

#!python -m spacy download de_core_news_lg
!python -m spacy download de_core_news_sm

#nlp = spacy.load("de_core_news_lg")
nlp = spacy.load("de_core_news_sm")
dic = pyphen.Pyphen(lang="de_DE")

def extract_haiku(sentence):
    s = sentence.strip()
    s = s.replace(",", "").replace(".", "").replace("!", "").replace("?", "").replace("-", "")
    if not re.fullmatch(r"[A-Za-z ]+", s):
        return None
    
    words = s.split()
    syllable_counts = [dic.inserted(w).count("-") + 1 for w in words]

    line_limits = [5, 7, 5]
    line_idx = 0
    line_sum = 0
    extracted_haiku = [[], [], []]

    for word, syllable_count in zip(words, syllable_counts):
        if line_idx >= 3 or line_sum > line_limits[line_idx]:
            break

        if line_sum < line_limits[line_idx]:
            extracted_haiku[line_idx].append(word)
            line_sum += syllable_count
            if line_sum == line_limits[line_idx]:
                line_idx += 1
                line_sum = 0

    haiku_word_count = sum(len(line) for line in extracted_haiku)
    if line_idx == 3 and line_sum == 0 and haiku_word_count == len(words):
        return [" ".join(line) for line in extracted_haiku]
    
    return None


haikus = []
context_indices = []
for doc in tqdm(nlp.pipe(wortmeldungen["text"].fillna("").astype(str), batch_size=50), total=len(wortmeldungen)):
    haikus.append([extract_haiku(sentence.text) for sentence in doc.sents])
    context_indices.append([(sentence.start_char, sentence.end_char) for sentence in doc.sents])
    
wortmeldungen["lines"] = haikus
wortmeldungen["context_indices"] = context_indices
wortmeldungen.head()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 9.0 MB/s  0:00:01m0:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


100%|██████████| 1000/1000 [00:48<00:00, 20.64it/s]


,period,session,date,speaker,text,lines,context_indices
55895,XXVI,15,2018-03-21T00:00:00,51565,Wir gelangen nun zu den Punkten 20 bis 23 der ...,"[None, None, None, None, None]","[(0, 115), (115, 170), (170, 216), (217, 226),..."
148799,XXIII,30,2007-07-06T00:00:00,1817,Nächste Rednerin ist Frau Abgeordnete Zwerschi...,"[None, None, None, None]","[(0, 49), (50, 93), (94, 103), (103, 111)]"
129279,XXIV,41,2009-10-22T00:00:00,2849,Hohes Haus! Meine Damen und Herren! Ich bin ic...,"[None, None, None, None, None, None, None, Non...","[(0, 11), (12, 35), (36, 206), (207, 428), (42..."
98299,XXIV,190,2013-02-19T00:00:00,3604,Der eingebrachte Antrag steht mit in Verhandlu...,"[None, None, None, None, None, None, None, Non...","[(0, 50), (50, 372), (373, 505), (505, 591), (..."
32247,XXVII,79,2021-01-20T00:00:00,35521,Der Abänderungsantrag wurde in den Grundzügen ...,"[None, None, None, None]","[(0, 150), (150, 237), (238, 266), (266, 274)]"


liste auf haikus flatten und contextBefore/after ableiten

In [5]:
haikus = wortmeldungen.explode(["lines", "context_indices"]).dropna(subset=["lines"])
haikus["line1"] = haikus["lines"].str[0]
haikus["line2"] = haikus["lines"].str[1]
haikus["line3"] = haikus["lines"].str[2]
haikus["context_before"] = haikus.apply(lambda row: row["text"][: row["context_indices"][0]], axis=1)
haikus["context_after"] = haikus.apply(lambda row: row["text"][row["context_indices"][1] :], axis=1)
haikus = haikus.drop(columns=["text", "context_indices", "lines"])
haikus.head()

,period,session,date,speaker,line1,line2,line3,context_before,context_after
17064,XXVII,174,2022-10-03T00:00:00,15526,Ich frage das weil,Sie mich gefragt haben ob,ich auf der Krim war,Herr Präsident! Meine sehr geehrten Damen und ...,"Ich frage Sie das jetzt hier, und die Österre..."
20053,XXVII,158,2022-05-19T00:00:00,35468,Der erste Stich hat,nicht gereicht sondern es hat,einen zweiten gebraucht,Meine sehr geehrten Damen und Herren vor den B...,"Wissen Sie, was dann passiert ist? – Zum drit..."
7144,XXVII,239,2023-11-21T00:00:00,2338,Mit diesem Budget,holen wir kein einziges,Kind aus der Armut,Herr Präsident! Herr Staatssekretär! Geschätzt...,(Abg. Sieber: Na geh! Sagt sogar der Budgetdi...
152308,XXIII,17,2007-03-29T00:00:00,14765,Deswegen ist es,wichtig dass wir da auf die,Wahlfreiheit setzen,Frau Präsidentin! Sehr geehrte Ministerinnen! ...,(Beifall bei der ÖVP.)\nDiese geplante Flexib...
110239,XXIV,130,2011-11-15T00:00:00,35468,Es gibt nicht eine,einzige Verurteilung,in diesem Bereich,Frau Präsident! Frau Bundesminister! Werte Kol...,"Stattdessen bilden wir Kommissionen, Experten..."


gesetzgebungsperiode als arabische zahl für tooltip

In [6]:
import roman

haikus["period_roman"] = haikus["period"]
haikus["period"] = haikus["period"].apply(lambda periodRoman: roman.fromRoman(periodRoman))
haikus.head()

,period,session,date,speaker,line1,line2,line3,context_before,context_after,period_roman
17064,27,174,2022-10-03T00:00:00,15526,Ich frage das weil,Sie mich gefragt haben ob,ich auf der Krim war,Herr Präsident! Meine sehr geehrten Damen und ...,"Ich frage Sie das jetzt hier, und die Österre...",XXVII
20053,27,158,2022-05-19T00:00:00,35468,Der erste Stich hat,nicht gereicht sondern es hat,einen zweiten gebraucht,Meine sehr geehrten Damen und Herren vor den B...,"Wissen Sie, was dann passiert ist? – Zum drit...",XXVII
7144,27,239,2023-11-21T00:00:00,2338,Mit diesem Budget,holen wir kein einziges,Kind aus der Armut,Herr Präsident! Herr Staatssekretär! Geschätzt...,(Abg. Sieber: Na geh! Sagt sogar der Budgetdi...,XXVII
152308,23,17,2007-03-29T00:00:00,14765,Deswegen ist es,wichtig dass wir da auf die,Wahlfreiheit setzen,Frau Präsidentin! Sehr geehrte Ministerinnen! ...,(Beifall bei der ÖVP.)\nDiese geplante Flexib...,XXIII
110239,24,130,2011-11-15T00:00:00,35468,Es gibt nicht eine,einzige Verurteilung,in diesem Bereich,Frau Präsident! Frau Bundesminister! Werte Kol...,"Stattdessen bilden wir Kommissionen, Experten...",XXIV


personen verknüpfen

In [7]:
personen = pd.read_json("woswormeileistung/data/persons.json")
personen = personen[["id", "name", "parties", "imageUrl"]]
personen = personen.rename(columns={"imageUrl": "image_url"})
personen["id"] = personen["id"].astype(str)
haikus = haikus.merge(personen, left_on="speaker", right_on="id", how="left")
haikus = haikus.drop(columns=["id"])
haikus = haikus.rename(columns={"name": "person_name", "speaker": "person_id"})
haikus["parties"] = haikus["parties"].apply(
    lambda x: x if isinstance(x, list) and len(x) > 0 else ["Ohne Klub"]
)
haikus.head()

,period,session,date,person_id,line1,line2,line3,context_before,context_after,period_roman,person_name,parties,image_url
0,27,174,2022-10-03T00:00:00,15526,Ich frage das weil,Sie mich gefragt haben ob,ich auf der Krim war,Herr Präsident! Meine sehr geehrten Damen und ...,"Ich frage Sie das jetzt hier, und die Österre...",XXVII,Dr. Reinhold Lopatka,[ÖVP],https://parlament.gv.at/dokument/bild/200695/2...
1,27,158,2022-05-19T00:00:00,35468,Der erste Stich hat,nicht gereicht sondern es hat,einen zweiten gebraucht,Meine sehr geehrten Damen und Herren vor den B...,"Wissen Sie, was dann passiert ist? – Zum drit...",XXVII,Dr. Dagmar Belakowitsch,[FPÖ],https://parlament.gv.at/dokument/bild/201579/2...
2,27,239,2023-11-21T00:00:00,2338,Mit diesem Budget,holen wir kein einziges,Kind aus der Armut,Herr Präsident! Herr Staatssekretär! Geschätzt...,(Abg. Sieber: Na geh! Sagt sogar der Budgetdi...,XXVII,Petra Wimmer,[SPÖ],https://parlament.gv.at/dokument/bild/200696/2...
3,23,17,2007-03-29T00:00:00,14765,Deswegen ist es,wichtig dass wir da auf die,Wahlfreiheit setzen,Frau Präsidentin! Sehr geehrte Ministerinnen! ...,(Beifall bei der ÖVP.)\nDiese geplante Flexib...,XXIII,Jochen Pack,[ÖVP],https://parlament.gv.at/dokument/bild/42059/42...
4,24,130,2011-11-15T00:00:00,35468,Es gibt nicht eine,einzige Verurteilung,in diesem Bereich,Frau Präsident! Frau Bundesminister! Werte Kol...,"Stattdessen bilden wir Kommissionen, Experten...",XXIV,Dr. Dagmar Belakowitsch,[FPÖ],https://parlament.gv.at/dokument/bild/201579/2...


möglichst stabile id spalte erzeugen

In [8]:
import hashlib

def hash(row):
    combined = (
        f"{row['period']} {row['session']} {row["line1"]} {row["line2"]} {row["line3"]}"
    )
    return hashlib.sha256(combined.encode()).hexdigest()

haikus["id"] = haikus.apply(hash, axis=1)
haikus.head()

,period,session,date,person_id,line1,line2,line3,context_before,context_after,period_roman,person_name,parties,image_url,id
0,27,174,2022-10-03T00:00:00,15526,Ich frage das weil,Sie mich gefragt haben ob,ich auf der Krim war,Herr Präsident! Meine sehr geehrten Damen und ...,"Ich frage Sie das jetzt hier, und die Österre...",XXVII,Dr. Reinhold Lopatka,[ÖVP],https://parlament.gv.at/dokument/bild/200695/2...,a84c729c71905fce00bb337d38212f53158923e8db6bc4...
1,27,158,2022-05-19T00:00:00,35468,Der erste Stich hat,nicht gereicht sondern es hat,einen zweiten gebraucht,Meine sehr geehrten Damen und Herren vor den B...,"Wissen Sie, was dann passiert ist? – Zum drit...",XXVII,Dr. Dagmar Belakowitsch,[FPÖ],https://parlament.gv.at/dokument/bild/201579/2...,520b1238e7710104d65a4464135e4fb38366ec810b579d...
2,27,239,2023-11-21T00:00:00,2338,Mit diesem Budget,holen wir kein einziges,Kind aus der Armut,Herr Präsident! Herr Staatssekretär! Geschätzt...,(Abg. Sieber: Na geh! Sagt sogar der Budgetdi...,XXVII,Petra Wimmer,[SPÖ],https://parlament.gv.at/dokument/bild/200696/2...,07cbcd0c172a7353edd5b8f77b86238221fd113aa39b87...
3,23,17,2007-03-29T00:00:00,14765,Deswegen ist es,wichtig dass wir da auf die,Wahlfreiheit setzen,Frau Präsidentin! Sehr geehrte Ministerinnen! ...,(Beifall bei der ÖVP.)\nDiese geplante Flexib...,XXIII,Jochen Pack,[ÖVP],https://parlament.gv.at/dokument/bild/42059/42...,09288f4294a9f98188422225d5691803643fd3c03dbf94...
4,24,130,2011-11-15T00:00:00,35468,Es gibt nicht eine,einzige Verurteilung,in diesem Bereich,Frau Präsident! Frau Bundesminister! Werte Kol...,"Stattdessen bilden wir Kommissionen, Experten...",XXIV,Dr. Dagmar Belakowitsch,[FPÖ],https://parlament.gv.at/dokument/bild/201579/2...,62dbd8df38c9a0b78cbd8dc206b874f9035d037f539c0a...


exportieren

In [9]:
haikus.to_json("web/haikus.json", orient="records", force_ascii=False, indent=2)